In [ ]:
import requests
import json
import os
import re
from tavily import TavilyClient
from IPython.display import display, Markdown, HTML, clear_output
import time
from openai import OpenAI

# 配置 API（改为 OpenAI）
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_BASE_URL = os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1")
MODEL_NAME = os.getenv("OPENAI_MODEL_ID", "gpt-4o-mini")

# 配置 Tavily（从环境变量读取）
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
if TAVILY_API_KEY:
    os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

print("API 配置完成!")

API 配置完成!


In [2]:
# 单元块 2: 定义工具函数
def get_weather(city: str) -> str:
    """查询城市天气"""
    url = f"https://wttr.in/{city}?format=j1"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        
        current_condition = data['current_condition'][0]
        weather_desc = current_condition['weatherDesc'][0]['value']
        temp_c = current_condition['temp_C']
        humidity = current_condition['humidity']
        wind_speed = current_condition['windspeedKmph']
        
        return f"{city}当前天气：{weather_desc}，气温{temp_c}°C，湿度{humidity}%，风速{wind_speed}km/h"
        
    except Exception as e:
        return f"错误：查询天气时遇到问题 - {e}"

def get_attraction(city: str, weather: str) -> str:
    """查询景点推荐"""
    try:
        tavily = TavilyClient(api_key=os.environ['TAVILY_API_KEY'])
        query = f"'{city}' 在'{weather}'天气下最值得去的旅游景点推荐及理由"
        
        response = tavily.search(query=query, search_depth="basic", include_answer=True)
        
        if response.get("answer"):
            return response["answer"]
        
        formatted_results = []
        for result in response.get("results", []):
            formatted_results.append(f"- {result['title']}: {result['content']}")
        
        if not formatted_results:
            return "抱歉，没有找到相关的旅游景点推荐。"

        return "根据搜索，为您找到以下信息：\n" + "\n".join(formatted_results)

    except Exception as e:
        return f"错误：执行搜索时出现问题 - {e}"

print("✅ 工具函数定义完成!")

✅ 工具函数定义完成!


In [ ]:
# 单元块 3: 定义智能助手类
class OpenAICompatibleClient:
    def __init__(self, model: str, api_key: str, base_url: str):
        self.model = model
        self.client = OpenAI(api_key=api_key, base_url=base_url)

    def generate(self, prompt: str, system_prompt: str) -> str:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt},
        ]
        resp = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            stream=False
        )
        return resp.choices[0].message.content

class NotebookTravelAssistant:
    def __init__(self, model_name):
        # 初始化 OpenAI 客户端
        try:
            self.client = OpenAICompatibleClient(
                model=model_name,
                api_key=os.getenv("OPENAI_API_KEY"),
                base_url=os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1"),
            )
        except Exception as e:
            print(f"⚠️ 模型 {model_name} 初始化失败: {e}，使用默认模型回退")
            self.client = OpenAICompatibleClient(
                model=os.getenv("OPENAI_MODEL_ID", "gpt-4o-mini"),
                api_key=os.getenv("OPENAI_API_KEY"),
                base_url=os.getenv("OPENAI_BASE_URL", "https://api.openai.com/v1"),
            )

        # 定义工具
        self.available_tools = {
            "get_weather": get_weather,
            "get_attraction": get_attraction,
        }

        # 系统提示词
        self.system_prompt = """
你是一个智能旅行助手。你的任务是分析用户的请求，并使用可用工具一步步地解决问题。

# 可用工具:
- `get_weather(city: str)`: 查询指定城市的实时天气。
- `get_attraction(city: str, weather: str)`: 根据城市和天气搜索推荐的旅游景点。

# 行动格式:
你的回答必须严格遵循以下格式。首先是你的思考过程，然后是你要执行的具体行动。
Thought: [这里是你的思考过程和下一步计划]
Action: [这里是你要调用的工具，格式为 function_name(arg_name="arg_value")]

# 任务完成:
当你收集到足够的信息，能够回答用户的最终问题时，你必须使用 `finish(answer="...")` 来输出最终答案。

请开始吧！
"""

    def generate_response(self, prompt: str) -> str:
        """调用 OpenAI 生成回应"""
        return self.client.generate(prompt, system_prompt=self.system_prompt)

    def parse_action(self, text: str):
        """解析行动指令"""
        action_match = re.search(r"Action: (.*)", text, re.DOTALL)
        if not action_match:
            return None, None

        action_str = action_match.group(1).strip()

        if action_str.startswith("finish"):
            final_answer_match = re.search(r'finish\(answer="(.*)"\)', action_str)
            if final_answer_match:
                return "finish", final_answer_match.group(1)
            return "finish", "抱歉，我无法提供完整的答案。"

        tool_match = re.search(r"(\w+)\((.*)\)", action_str)
        if not tool_match:
            return None, None

        tool_name = tool_match.group(1)
        args_str = tool_match.group(2)

        kwargs = {}
        if args_str:
            arg_matches = re.findall(r'(\w+)="([^"]*)"', args_str)
            kwargs = dict(arg_matches)

        return tool_name, kwargs

print("✅ 智能助手类定义完成!")

✅ 智能助手类定义完成!


In [5]:
# 单元块 4: 定义显示函数
def display_user_input(user_input):
    """显示用户输入"""
    display(HTML(f"""
    <div style="background: #f0f7ff; border-left: 4px solid #4dabf7; padding: 12px 16px; margin: 10px 0; border-radius: 4px;">
        <div style="font-weight: bold; color: #1971c2; margin-bottom: 4px;">👤 用户:</div>
        <div>{user_input}</div>
    </div>
    """))

def display_cycle_header(cycle_num):
    """显示循环标题"""
    display(HTML(f"""
    <div style="background: #fff3bf; border-left: 4px solid #fcc419; padding: 8px 12px; margin: 10px 0; border-radius: 4px;">
        <strong>🔄 循环 {cycle_num}:</strong>
    </div>
    """))

def display_thought(thought_text):
    """显示思考过程"""
    display(HTML(f"""
    <div style="background: #e6fcf5; border-left: 4px solid #20c997; padding: 12px 16px; margin: 8px 0; border-radius: 4px;">
        <div style="font-weight: bold; color: #087f5b; margin-bottom: 4px;">🧠 智能体响应:</div>
        <div><strong>Thought:</strong> {thought_text}</div>
    </div>
    """))

def display_action(tool_name, args):
    """显示行动"""
    display(HTML(f"""
    <div style="background: #fff9db; border-left: 4px solid #f59f00; padding: 8px 12px; margin: 8px 0; border-radius: 4px;">
        <strong>⚡ 执行工具:</strong> {tool_name}({', '.join([f'{k}="{v}"' for k, v in args.items()])})
    </div>
    """))

def display_observation(observation):
    """显示观察结果"""
    display(HTML(f"""
    <div style="background: #f8f9fa; border-left: 4px solid #adb5bd; padding: 12px 16px; margin: 8px 0; border-radius: 4px;">
        <div style="font-weight: bold; color: #495057; margin-bottom: 4px;">👁️ 观察结果:</div>
        <div>{observation}</div>
    </div>
    """))

def display_final_result(result):
    """显示最终结果"""
    display(HTML(f"""
    <div style="background: #d3f9d8; border: 2px solid #40c057; padding: 16px 20px; margin: 16px 0; border-radius: 8px;">
        <div style="font-weight: bold; color: #2b8a3e; font-size: 18px; margin-bottom: 8px;">✅ 任务完成!</div>
        <div style="font-size: 16px; line-height: 1.5;">{result}</div>
    </div>
    """))

print("✅ 显示函数定义完成!")

✅ 显示函数定义完成!


In [6]:
# 单元块 5: 定义主运行函数
def run_assistant(user_input, max_iterations=5):
    """运行智能助手"""
    # 创建助手实例
    assistant = NotebookTravelAssistant(model_name=MODEL_NAME)
    
    # 显示用户输入
    display_user_input(user_input)
    
    prompt_history = [f"用户请求: {user_input}"]
    
    for i in range(max_iterations):
        # 显示循环标题
        display_cycle_header(i+1)
        
        # 调用模型
        full_prompt = "\n".join(prompt_history)
        llm_output = assistant.generate_response(full_prompt)
        
        # 提取思考过程
        thought_match = re.search(r"Thought: (.*?)(?=Action:|$)", llm_output, re.DOTALL)
        thought_text = thought_match.group(1).strip() if thought_match else "思考过程未明确显示"
        
        # 显示思考过程
        display_thought(thought_text)
        
        # 解析并执行行动
        tool_name, args = assistant.parse_action(llm_output)
        
        if tool_name == "finish":
            # 显示最终结果
            display_final_result(args)
            return args
        
        elif tool_name in assistant.available_tools:
            # 显示行动
            display_action(tool_name, args)
            
            # 执行工具
            observation = assistant.available_tools[tool_name](**args)
            
            # 显示观察结果
            display_observation(observation)
            
            # 更新历史
            prompt_history.append(llm_output)
            prompt_history.append(f"Observation: {observation}")
            
            # 添加短暂延迟，使输出更易读
            time.sleep(1)
        else:
            display(HTML(f"<div style='color: red; padding: 10px;'>❌ 无法解析行动: {tool_name}</div>"))
            break
    
    return "抱歉，我无法完成这个请求。请尝试更具体的查询。"

print("✅ 主运行函数定义完成!")

✅ 主运行函数定义完成!


In [7]:
# 单元块 6: 运行示例
user_input = "你好，请帮我查询一下今天北京的天气，然后根据天气推荐一个合适的旅游景点"
result = run_assistant(user_input)

InvalidArgument: 400 API key not valid. Please pass a valid API key. [reason: "API_KEY_INVALID"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "generativelanguage.googleapis.com"
}
, locale: "en-US"
message: "API key not valid. Please pass a valid API key."
]

In [40]:
# 单元块 7: 实现交互式对话功能
def interactive_travel_assistant():
    """交互式旅行助手"""
    print("🌍 欢迎使用智能旅行助手!")
    print("=" * 50)
    print("我可以帮您查询任何城市的天气，并根据天气推荐合适的旅游景点。")
    print("您只需输入城市名称，我将为您提供完整的旅行建议。")
    print("输入 '退出'、'quit' 或 'exit' 可以结束对话。")
    print("=" * 50)
    
    conversation_count = 0
    
    while True:
        # 获取用户输入
        user_input = input(f"\n🏙️  请输入城市名称 (对话 {conversation_count + 1}): ").strip()
        
        # 检查退出条件
        if user_input.lower() in ['退出', 'quit', 'exit']:
            print("\n" + "=" * 50)
            print("👋 感谢使用智能旅行助手，期待下次为您服务!")
            print("=" * 50)
            break
            
        # 检查空输入
        if not user_input:
            print("⚠️  请输入有效的城市名称。")
            continue
        
        # 构建查询
        query = f"请帮我查询{user_input}的天气，然后根据天气推荐合适的旅游景点"
        
        print(f"\n📡 处理您的请求: {query}")
        print("-" * 50)
        
        # 运行助手
        result = run_assistant(query)
        
        # 增加对话计数
        conversation_count += 1
        
        print("\n" + "=" * 50)
        print(f"💫 已完成 {conversation_count} 次对话")
        print("=" * 50)

# 启动交互式助手
print("✅ 交互式对话功能已加载!")
print("运行 'interactive_travel_assistant()' 开始对话")

✅ 交互式对话功能已加载!
运行 'interactive_travel_assistant()' 开始对话


In [41]:
# 启动交互式对话
interactive_travel_assistant()

🌍 欢迎使用智能旅行助手!
我可以帮您查询任何城市的天气，并根据天气推荐合适的旅游景点。
您只需输入城市名称，我将为您提供完整的旅行建议。
输入 '退出'、'quit' 或 'exit' 可以结束对话。



🏙️  请输入城市名称 (对话 1):  伦敦



📡 处理您的请求: 请帮我查询伦敦的天气，然后根据天气推荐合适的旅游景点
--------------------------------------------------



💫 已完成 1 次对话



🏙️  请输入城市名称 (对话 2):  郑州


...


💫 已完成 2 次对话



🏙️  请输入城市名称 (对话 3):  莆田



📡 处理您的请求: 请帮我查询莆田的天气，然后根据天气推荐合适的旅游景点
--------------------------------------------------



💫 已完成 3 次对话


KeyboardInterrupt: Interrupted by user